In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241212_02_MLlib_Regression").getOrCreate()
spark

24/12/12 15:48:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/12 15:48:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/12 15:48:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
train_df = spark.read.format("csv")\
    .option("header",'true')\
    .option('inferSchema', 'true')\
    .load("data/house_train.csv")

test_df = spark.read.format("csv")\
    .option("header",'true')\
    .option('inferSchema', 'true')\
    .load("data/house_test.csv")

In [12]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [13]:
test_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [8]:
train_df.show(), test_df.show()

24/12/12 15:51:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

(None, None)

In [9]:
# sale price : Label
# features: 

# 전처리 : 결측치 = 0  

- 숫자 타입 통일 

In [14]:
# 결측치 0 
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

# Encoding 
- 문자형 > 숫자형 1,2,3,4

In [61]:
# 데이터 타입 일치 시키기 
# train_df 의 GarageCars 와 GarageArea 컬럼은 integer 인데 test_df 는 string 임 

from pyspark.sql.functions import col
# GarageCars, GarageArea 컬럼을 정수형으로 변환
test_df = test_df.withColumn("GarageCars", col("GarageCars").cast("integer"))
test_df = test_df.withColumn("GarageArea", col("GarageArea").cast("integer"))

In [62]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

#1,2,3,4 로 값을 단순화
string_columns = ["Neighborhood"]
indexers = [StringIndexer(inputCol=col, outputCol= col+"_index") for col in string_columns]

In [63]:
# onehotencoding : 범주형 변수 1,2,3,4 -> 1로 바꾸는 인코딩
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol= col+"_encoded") for col in string_columns]

# Features selection

In [64]:
numeric_columns = ["LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

In [65]:
assembler_inputs = [col+"_encoded" for col in string_columns] + numeric_columns

# Assembler

In [66]:
assembler = VectorAssembler(inputCols= assembler_inputs, outputCol="features")

# Label selection

In [67]:
train_df = train_df.withColumnRenamed("SalePrice", "label")

In [68]:
train_df.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|B

# Pipeline 설정
- StringIndex + OneHotEncoder + Assembler >> 하나의 SparkML Pipeline 으로 결합

In [69]:
from pyspark.ml import Pipeline

In [70]:
#pipeline 생성
pipeline = Pipeline(stages = indexers+encoders+[assembler])

In [71]:
#pipeline 실행
pipeline_model = pipeline.fit( train_df)

In [72]:
train_transformed = pipeline_model.transform(train_df)

# 예측 > 회귀 > 모델 학습 > 평가 > 예측

In [73]:
from pyspark.ml.regression import LinearRegression

In [74]:
lr = LinearRegression(featuresCol="features",labelCol="label")

# 학습된 모델
lr_model = lr.fit(train_transformed)

24/12/12 17:31:33 WARN Instrumentation: [1aed690c] regParam is zero, which might cause numerical instability and overfitting.


In [75]:
# 평가 - 데이터를 이용한 평가 > FIT 은 안함 - FIT 은 모델 맞춤 과정, 테스트 데이터에 의해 규칙이 변화
test_transformed = pipeline_model.transform(test_df)
predictions = lr_model.transform(test_transformed)

In [ ]:
# 예측